In [0]:
import numpy as np 
import os 
###https://www.geeksforgeeks.org/working-zip-files-python/
##import zip code from here

# importing required modules 
from zipfile import ZipFile 


  
# opening the zip file in READ mode 
with ZipFile('/content/drive/My Drive/20_newsgroups.zip', 'r') as zip: 
    # printing all the contents of the zip file 
    # zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall()
    print('Done!')

Extracting all the files now...
Done!


In [0]:
!pip install num2words

     |████████████████████████████████| 102kB 2.5MB/s 


Library:

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 18 20:46:09 2020
MT19049
@author: akanksha
"""

from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from num2words import num2words
import re
import string
import math
import random

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Preprocessing:

In [0]:



def preprocessing(text):
    token_list=[]
    temp = re.findall(r'\d+', text) 
    res = list(map(int, temp)) 
    s=""
    for i in res:
        
        s=s+num2words(i)+" "
    text=text+" "+s
    
    text=text.strip()
    table = str.maketrans('', '', string.punctuation)
    
    text = text.translate(table)
    text=re.sub(r'\d+','',text)
    token_list=word_tokenize(text)
    stop_words=set(stopwords.words('english'))
    token_list=[token for token in token_list if not token in stop_words]
    token_list=[token.lower() for token in token_list]
    ps=PorterStemmer()
    token_list=[ps.stem(token) for token in token_list]
    
    return token_list


build all the dictionary:

In [0]:
# all the dictionary are initialised
import os
folders=os.listdir("/content/20_newsgroups/")
folders=['comp.graphics','sci.med','talk.politics.misc','rec.sport.hockey','sci.space']

document_name_id={}
document_terms={}
classwise_dictionary={}
classwise_dictionary_doc={}
doc_id=-1
vocabulary=[]
voc=-1
for folder in folders:
    print(folder)
    fpath='/content/20_newsgroups/'+ folder
    if(os.path.isdir(fpath)):
        file_names=os.listdir("/content/20_newsgroups/" + folder)
        for file_name in file_names:
            classwise_dictionary[folder]=[]
            classwise_dictionary_doc[folder]=[]
            doc_id+=1
            document_name_id[doc_id]=[]
            document_name_id[doc_id].append(folder)
            document_name_id[doc_id].append(file_name)
            #file read karaaaa readFile("Data/" + folder+"/"+file_name)
            f = open("/content/20_newsgroups/" + folder+"/"+file_name, 'r',encoding ="ascii", errors ="surrogateescape")
           
            text=f.read()
            f.close()
           
            term_list=list(preprocessing(text))
            vocabulary=list(set(vocabulary+term_list))
            document_terms[doc_id]=term_list
            
            

comp.graphics
sci.med
talk.politics.misc
rec.sport.hockey
sci.space


test train split:

In [0]:
# aplit of test and train data

def split(train,test):
  document_id=np.asarray(list(document_name_id.keys()))
  random.shuffle(document_id)
  len(document_id)
  train_=document_id[:int(len(document_id)*train)]
  test_=document_id[int(len(document_id)*train):]
  return test_,train_

In [0]:
# assignment  of all the documents and merging of all the tokens classwise.
def classwise_assignment(train_data):
  voc=[]
  for i in train_:
    print(i)
    if document_name_id[i][0] in classwise_dictionary.keys():
      classwise_dictionary_doc[document_name_id[i][0]].append(i)
      classwise_dictionary[document_name_id[i][0]]=classwise_dictionary[document_name_id[i][0]]+document_terms[i]
      voc=list(set(voc+classwise_dictionary[document_name_id[i][0]]))
  return voc

MI

In [0]:
# class_wise_term_MI_formula
def sort_dictionary(x):
  for i in x.keys():
    class_wise_term_MI_formula[i]={k: v for k, v in sorted(x[i].items(), key=lambda item: item[1] ,reverse=True)}


In [0]:
# inplace of 0.2 or 0.2 u have to p[ut train:test split ratio]
# split test train
test_,train_=split(0.8,0.2)

TF IDF  feature selection:

In [0]:
# tf-idf dictionary initialised
class_wise_term_tfidf={}
for i in classwise_dictionary.keys():
  class_wise_term_tfidf[i]={}

In [0]:
# df count of all the train unique terms class wise.
def df_count(classwise_dictionary,term):
  c=0
  for i in classwise_dictionary:
    
    if term in  classwise_dictionary[i]:
      c=c+1
  return c

In [0]:
# calculate the tf-idf counts in dictionary of all the terms in train set.
def feature_selection_tfidf(voc):
  for i in class_wise_term_tfidf.keys():

    for j in voc:
        print(j)
        class_wise_term_tfidf[i][j]=[]
        class_wise_term_tfidf[i][j].append(classwise_dictionary[i].count(j))
        class_wise_term_tfidf[i][j].append(df_count(classwise_dictionary,j))



In [0]:
# tf-idf calculation with formula.
def tf_idf_calculation(class_wise_term_tfidf):
  for i in class_wise_term_tfidf.keys():
    for j in class_wise_term_tfidf[i]:
       a=class_wise_term_tfidf[i][j][0]
       c=class_wise_term_tfidf[i][j][1]
       d=len(class_wise_term_tfidf.keys())
       if a!=0 and d!=0 and c!=0: 
          class_wise_term_tfidf[i][j]=(1+math.log10(a))*math.log10(d/c)
       else:
           class_wise_term_tfidf[i][j]=0

In [0]:
# sorrt the dictionary of tf-idf calculation.
def sort_dictionary_tf_idf(x):
  for i in x.keys():
    class_wise_term_tfidf[i]={k: v for k, v in sorted(x[i].items(), key=lambda item: item[1] ,reverse=True)}


In [15]:
# function for tf df count.
voc=classwise_assignment(train_)
feature_selection_tfidf(voc)

Streaming output truncated to the last 5000 lines.
overutil
shoumidwayuchicagoedu
cantaloupesrvcscmueducrabapplesrvcscmuedufsececmuedueuropaenggtefsdcomhowlandrestonansnetbogussuranetnewsfeedpeachneteduumnedulynxunmeducarinaunmedubhjel
weston
tint
cantaloupesrvcscmuedudasnewsharvardedunocnearnethowlandrestonansnetzaphodmpsohiostateedudarwinsuranetuvaarpavdoeghast
jasondaw
bldg
outin
whileadmittedli
deficitdark
bowel
sweeten
diciplin
edofafidonetorg
piclab
voter
finer
cfkbexcscmuedu
fahrenheit
solklar
shinni
fortran
parentpar
cantaloupesrvcscmuedurochesterudelbogussuranetnewsfeedpeachneteduconcertsambausenet
malariaendem
uab
sator
aqtarz
rkoepmnetworkucsdedu
anythink
intel
instend
cantaloupesrvcscmuedudasnewsharvardedunocnearnethowlandrestonansnetzaphodmpsohiostateeduuwmedulinacuchinewskimbarkthf
fpermanetorg
cantaloupesrvcscmuedumagnesiumclubcccmuedupittedugatechhowlandrestonansnetagateamessunbarrshwidewnoctyonewscstitechsystitechacjppititechpisunmalek
ifn
gatechwameikezvgari
understan

In [0]:

tf_idf_calculation(class_wise_term_tfidf)

In [0]:
# topk words which are slecetd are kept here
topk_tfidf={}
for i in classwise_dictionary.keys():
  topk_tfidf[i]={}

In [0]:
sort_dictionary_tf_idf(class_wise_term_tfidf)

In [0]:
def tf_idf_topk_terms(k,class_wise_term_tfidf):
  for i in class_wise_term_tfidf.keys():
    b=k
    
    for j in class_wise_term_tfidf[i]:
      if b!=0:
        print(b)
        b=b-1
        topk_tfidf[i][j]=class_wise_term_tfidf[i][j]

In [20]:
# call for selecting top k fwature here in place of 1000 u can put ur k feature.
tf_idf_topk_terms(1000,class_wise_term_tfidf)

Streaming output truncated to the last 5000 lines.
999
998
997
996
995
994
993
992
991
990
989
988
987
986
985
984
983
982
981
980
979
978
977
976
975
974
973
972
971
970
969
968
967
966
965
964
963
962
961
960
959
958
957
956
955
954
953
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
920
919
918
917
916
915
914
913
912
911
910
909
908
907
906
905
904
903
902
901
900
899
898
897
896
895
894
893
892
891
890
889
888
887
886
885
884
883
882
881
880
879
878
877
876
875
874
873
872
871
870
869
868
867
866
865
864
863
862
861
860
859
858
857
856
855
854
853
852
851
850
849
848
847
846
845
844
843
842
841
840
839
838
837
836
835
834
833
832
831
830
829
828
827
826
825
824
823
822
821
820
819
818
817
816
815
814
813
812
811
810
809
808
807
806
805
804
803
802
801
800
799
798
797
796
795
794
793
792
791
790
789
788
787
786
785
784
783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
7

In [0]:

# idf calculation of terms:in all train set
idf_={}
for j in voc:
  for i in document_terms:
    if j in document_terms[i] and j in idf_.keys():
      idf_[j]+=1
    else:
      if j in document_terms[i] and j not in idf_.keys():
        idf_[j]=1



In [0]:
# vector formation of all the words classwise.
def make_doc_vector_train_doc():
  doc_vector={}
  # class_dic_class===key comp.graph:
  for k,v in classwise_dictionary_doc.items():
    for l in v:
      doc_vector[l]={}
      for j in topk_tfidf[k]:
        if j in document_terms[l]:
          doc_vector[l][j]=document_terms[l].count(j)*idf_[j]
        else:
          doc_vector[l][j]=0

  return doc_vector

In [0]:
vectors_train=make_doc_vector_train_doc()

In [0]:
# for predicting one test document with all train documemnts and 
def test_prediction(vector_train,doc_test,k):
  s=0
  predict={}
  for i in vectors_train:
    s=0
    t=0
    for j in vectors_train[i]:
      if j in doc_test:
        s+=vectors_train[i][j]
        t+=(vectors_train[i][j])**2
      else:
        s+=0
        t+=0
    if t!=0:
      predict[i]=s/math.sqrt(t)
    else:
      predict[i]=0
  predict={k: v for k, v in sorted(predict.items(), key=lambda item: item[1],reverse=True)}
 
  result=[]
  for i in predict:
    if k==0:
      return max(set(result), key =result.count) 
    else:
      result.append(document_name_id[i][0])
      k=k-1 

In [0]:
# calling for predicting each test dcs.
from scipy import stats as s
def knn():
  predict={}
  for i in test_:
    predict[i]=test_prediction(vectors_train,document_terms[i],5)
  return predict

In [0]:
nn=0
predict=knn()
for i in test_:
  if predict[i]==document_name_id[i][0]:
    nn=nn+1
print(nn/len(test_))

In [0]:
# confusion matrix is created.
nm=0
Y_actual=[]
Y_predict=[]
for i in test_:
    Y_actual.append(document_name_id[i][0])
    Y_predict.append(predict[i])
    if document_name_id[i][0]==predict[i]:
       nm+=1
print(nm/len(test_))
from sklearn.metrics import confusion_matrix 
results = confusion_matrix(Y_actual, Y_predict) 
print(results)